geotiff_rain.py                                                     <br>
                                                                    <br>
 This code converts total column water vapor (TotalColWaterVapor)   <br>
   in the AMSR-2 NRT Rain product in HDF-EOS5 format to GeoTiff,    <br>
   with the use of GIBS color scheme for the variable.              <br>
                                                                    <br>
Use remapSwath() to remap data array to (epsg 4326) projection      <br>
   i.e. WGS84.  orient='slant' option puts the  projected GeoTiff   <br>
   the slanted swath of the original data that requires less        <br>
   not-value fill-in pixels, i.e. upscaling.                        <br>
                                                                    <br>
 User should use <latRange> to select the latitude range of the     <br>
   swath (based on the center line of the swath) to be converted to <br>
   geotiff. For various reasons, it is recommended the range to be  <br>
   limited to any numbers between -75 and 75 degrees.               <br>
                                                                    <br>
 The output Geotiff imagery "looks" retangular as GeoTiff imageries <br>
   always are before loaded to a map, i.e. it will appear slanted   <br>
      along the swath direction with map projection.                <br>
                                                                    <br>
 Input: AMSR-2 NRT Rain file                                        <br>
 Output: GeoTIFF file in of selected latitude range of the Rain     <br>
         product swath in epsg4326 (WGS 84) projection with given   <br>
         geotransformation parameters in the output file            <br>
                                                                    <br>
 Required Python packages:                                          <br>
     numpy, scipy, pyproj, gdal and GHRC's GIBScolors_utils         <br>
                                                                    <br>
 Notes:                                                             <br>
   Original data type for 'TotalColWaterVapor' is byte (-127 to 128)<br>
   It's converted to unsigned byte (0-255) for the GeoTiff output   <br>
                                                                    <br>
   User can set <useGIBScolors> to True/False to elect use of GIBS  <br>
   color scheme or not.                                             <br>
   In the case of useGIBScolors=True, <gibsMapDir> the location     <br>
   of GIBS color scheme files needs to be provided                  <br>
                                                                    <br>
 By Y. WU of AMSR team @GHRC, Aug 2021                             <br>

In [ ]:
! pip install numpy
! conda install -c conda-forge gdal

import numpy as np
from osgeo import gdal, osr
from ipynb.fs.full.ConvSwath import remapSwath
from ipynb.fs.full.GIBScolors_utils import *

#---- File name and initial run parameters
h5file = 'test_files/geotiff/AMSR_U2_L2_Rain_R01_202108041504_A.he5'
fileParts =h5file.split('_')
tstr = fileParts[-2]

#--- latitude range for the SWATHS data to be converted to GeoTiff
#    inclusion of higher latitudes could greatly increase upscaling, as well as process time 
latRange = [-60,60]

#--- name of variable to be converted
vname = 'TotalColWaterVapor'

#--- fillv for the variable in the original AMSR file 
fillv = -99

#--- Option to use GIBS color scheme
useGIBScolors = True
gibsMapDir='test_files/color_map/' #<-- ends with '/' for colormap files parsing

#--- Open hdf5 with GDAL and get data attributes
hdf_ds = gdal.Open(h5file, gdal.GA_ReadOnly)
dslist = hdf_ds.GetSubDatasets()
ds1 = [s[0].split('//HDFEOS/SWATHS/')[-1] for s in dslist] # include lat, lon

#---get data from the original hdf dataset
for ii, a in enumerate(ds1):
    if(vname in a): isubset = ii
varband = gdal.Open(hdf_ds.GetSubDatasets()[isubset][0], gdal.GA_ReadOnly)
bandArr0 = varband.ReadAsArray()

#---get (lat, lon) for the grid and geotransform info
for ii, a in enumerate(ds1):
    if('/Lat' in a): ilat = ii
    if('/Lon' in a): ilon = ii

Lat = gdal.Open(hdf_ds.GetSubDatasets()[ilat][0], gdal.GA_ReadOnly)
Lon = gdal.Open(hdf_ds.GetSubDatasets()[ilon][0], gdal.GA_ReadOnly)
Lat = Lat.ReadAsArray()
Lon = Lon.ReadAsArray()

#---remap data array and obtain geotransform parameters
bandArr, geotransform = remapSwath(bandArr0, Lon, Lat, latRange, fillv=fillv, orient='slant')

NY,NX = bandArr.shape

#---fix band description for output img size/shape
banddsc = vname + '('+str(NY)+'x'+str(NX)+') not org.'

#-----------------------------------------------
#- set driver, datatype, projection, and       -
#-  geotransformation for output geotiff       -
#-----------------------------------------------
#<--- output geotiff file name
TIFFpath= 'test_files/out/'+vname+'_'+tstr+'.tif'
driver = gdal.GetDriverByName('GTiff')
out_ds = driver.Create(TIFFpath,
                       NX, NY,
                       1,
                       gdal.GDT_Byte)


OutPrj = osr.SpatialReference()
OutPrj.ImportFromEPSG(4326)  #<--use (lat,lon)
out_ds.SetProjection(OutPrj.ExportToWkt())
out_ds.SetGeoTransform(geotransform)

#---activate output band
outBand = out_ds.GetRasterBand(1)
notVal = 255

#--set not-value and data type
bandArr[bandArr==fillv] = notVal


#---make geotiff with GIBS colors if so opted
outBand = out_ds.GetRasterBand(1)
scaleFC = {}
if(useGIBScolors): 
    gibsColors = gibsScheme(gibsMapDir, 'Rain', vname, scaleFC)
    if gibsColors:
        colors = colorTable(gibsColors, 255, scaleFC, verb=False)
        outBand.SetColorTable(colors)
        outBand.SetColorInterpretation(gdal.GCI_PaletteIndex)

#---writing geotiff band
bandArr = bandArr.astype('uint8')
outBand.WriteArray(bandArr)
outBand.SetDescription(banddsc)
outBand.SetNoDataValue(notVal)

outBand = None
out_ds = None  
print('Save to',TIFFpath,'\n')
